# 📊 **Antigranular** Heart Disease Prediction Contest (ft. **Harvard/OpenDP** and **TPDP**)

🎉 Welcome to a new [Antigranular](https://antigranular.com) contest in collaboration with the [TPDP Workshop](https://tpdp.journalprivacyconfidentiality.org/2024/) and [Harvard's OpenDP Community Meeting](https://opendp.org/)!

🩺 This time, we are focusing on [heart condition detection](https://en.wikipedia.org/wiki/Cardiovascular_disease) using our new [TensorFlow Privacy](https://github.com/tensorflow/privacy) and [Opacus (PyTorch)](https://opacus.ai/) models!

🦜 Any questions? Head over to our [Discord](https://discord.com/invite/KJwApgXs4s)!



## 🏃‍♂️ Getting Started

In this section we will download the antigranular package and login




### 📦 Install Antigranular

This command installs the [Antigranular PyPI Package](https://pypi.org/project/antigranular/) on the local enviroment.


In [ ]:
# Install the Antigranular package
!pip install antigranular &> /dev/null

### ✍ Login to the Enclave

Head over to [Competitions](https://www.antigranular.com/competitions) to find your `<user_id>`, `<user_secret>` and the competition's name and copy that command here.

![img](https://docs.antigranular.com/shots/comp_cell.png)

In [ ]:
import antigranular as ag
session = ag.login(<client_id>,<client_secret>, competition = "Heart Disease Prediction Hackathon")

Dataset "Heart Disease Prediction Hackathon Dataset" loaded to the kernel as heart_disease_prediction_hackathon_dataset
Key Name                       Value Type     
---------------------------------------------
train_y                        PrivateDataFrame
train_x                        PrivateDataFrame
test_x                         DataFrame      

Connected to Antigranular server session id: 061bb219-76f1-46bd-abd9-511c9f33e345, the session will time out if idle for 25 minutes
Cell magic '%%ag' registered successfully, use `%%ag` in a notebook cell to execute your python code on Antigranular private python server
🚀 Everything's set up and ready to roll!


### 🤖 Using AG

You can now simply use ``%%ag`` to run code on an enclave! You can always head over to our [Docs](https://docs.antigranular.com/) to learn more about AG, but for now, we can define train and test variables as follows.

In [ ]:
%%ag
x_train = heart_disease_prediction_hackathon_dataset["train_x"]
y_train = heart_disease_prediction_hackathon_dataset["train_y"]
x_test = heart_disease_prediction_hackathon_dataset["test_x"]

### 🕵️‍♂️ Exploring data

Exploring data in Antigranular involves spending your epsilon budget, be mindful of your usage but remember that the less epsilon you use, the less accurate your results will get!

In [ ]:
%%ag
x_train.info()

+----+----------+-------------+---------------+---------+------------+
|    | Column   | numerical   | categorical   | dtype   | bounds     |
|----+----------+-------------+---------------+---------+------------|
|  0 | age      | True        | False         | int64   | (21, 86)   |
|  1 | sex      | True        | False         | int64   | (0, 1)     |
|  2 | bp       | True        | False         | int64   | (80, 215)  |
|  3 | ch       | True        | False         | int64   | (102, 597) |
|  4 | bs       | True        | False         | int64   | (67, 157)  |
|  5 | phr      | True        | False         | int64   | (62, 222)  |
+----+----------+-------------+---------------+---------+------------+



In [ ]:
%%ag
y_train.info()

+----+-----------+-------------+---------------+---------+----------+
|    | Column    | numerical   | categorical   | dtype   | bounds   |
|----+-----------+-------------+---------------+---------+----------|
|  0 | condition | True        | False         | int64   | (0, 1)   |
+----+-----------+-------------+---------------+---------+----------+



In [ ]:
%%ag
# x_test is a public test set, so we can print it without using epsilon
ag_print(x_test)

      age  sex   bp   ch   bs  phr
0      71    1  128  326   95  117
1      61    1  153  270   98  123
2      59    1  113  236  106  181
3      69    0  109  151  109  108
4      55    0  137  235  101  150
...   ...  ...  ...  ...  ...  ...
1995   60    1  128  261  112  143
1996   50    1  143  216   94  100
1997   64    1  120  172   87  142
1998   56    1  158  294   82  144
1999   69    0  117  559  112  157

[2000 rows x 6 columns]



### 🎈 A quick solution

In this section we evaluate an editorial solution in AG using TensorFlow!

In [ ]:
%%ag
import tensorflow as tf
from op_pandas import standard_scaler, PrivateDataFrame
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from op_tensorflow import PrivateKerasModel, PrivateDataLoader


# Normal keras model
seqM = Sequential([
    Dense(64, activation='relu', input_shape=(6,)),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')  # Binary classification
])

# Create DP keras model
dp_model = PrivateKerasModel(model=seqM, l2_norm_clip=1, noise_multiplier=1)

# Use a standard (non-DP) optimizer directly from keras.
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

# PrivateKerasModel uses similar API as standard Keras
dp_model.compile(
	optimizer = optimizer,
	loss = 'binary_crossentropy',
	metrics = ["accuracy"]
)

/usr/local/lib/python3.10/site-packages/tensorflow_probability/python/__init__.py:57: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if (distutils.version.LooseVersion(tf.__version__) <
/usr/local/lib/python3.10/site-packages/tensorflow_probability/python/__init__.py:58: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  distutils.version.LooseVersion(required_tensorflow_version)):



In [ ]:
%%ag
# Use a standard (non-DP) optimizer directly from keras.
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

# PrivateKerasModel uses similar API as standard Keras
dp_model.compile(
	optimizer = optimizer,
	loss = 'binary_crossentropy',
	metrics = ["accuracy"]
)

In [ ]:
%%ag
x_train_scaled = standard_scaler(x_train, eps=2) # araştırılacak 1 2 3 vs vs
x_train_scaled.info()

+----+----------+-------------+---------------+---------+------------------------------------------+
|    | Column   | numerical   | categorical   | dtype   | bounds                                   |
|----+----------+-------------+---------------+---------+------------------------------------------|
|  0 | age      | True        | False         | float64 | (-3.6896648975857778,                    |
|    |          |             |               |         | 3.4404975950814625)                      |
|  1 | sex      | True        | False         | float64 | (-1.4326773757478135,                    |
|    |          |             |               |         | 0.6941300635406028)                      |
|  2 | bp       | True        | False         | float64 | (-2.991646146507617, 4.815826183892343)  |
|  3 | ch       | True        | False         | float64 | (-2.703254171243013, 6.54649771295959)   |
|  4 | bs       | True        | False         | float64 | (-2.178042393113138, 3.2769398479

In [ ]:
%%ag
data_loader = PrivateDataLoader(feature_df=x_train_scaled , label_df=y_train, batch_size=2) # batch size düşürülebilir 3 - '2

In [ ]:
%%ag
dp_model.fit(x=data_loader, epochs=5, target_delta=1) #1-e5 ? earlystop  '1 olabilir  - değiştirilebilir accuracy e göre 10 '20

Epoch 1/5

4000/4000 - 59s - loss: 0.0507 - accuracy: 0.9815 - 59s/epoch - 15ms/step

Epoch 2/5

4000/4000 - 60s - loss: 0.0568 - accuracy: 0.9800 - 60s/epoch - 15ms/step

Epoch 3/5



Kernel interrupted successfully


In [ ]:
%%ag
x_test_scaler = standard_scaler(PrivateDataFrame(x_test), eps=2) #epsilon
y_pred = dp_model.predict(x_test_scaler, label_columns=["output"])

63/63 [==============================] - 0s 5ms/step



In [ ]:
%%ag
# Note that the predictions are a float scalar
# so we scale it
def f(x: float) -> float:
  if x > 0.5:
    return 1
  else:
    return 0

y_pred["output"] = y_pred["output"].map(f, output_bounds=(0, 1))

### 📝 Make your submission (Through AG)

Submit a prediction by simply typing `submit_predictions(your_prediction)` to find out how you rank on the leaderboard.

![img](https://www.antigranular.com/static/media/Step%209.8091828f3cff4324fe6d.png)


In [ ]:
%%ag
result = submit_predictions(y_pred)

score: {'leaderboard': 0.894437790419707, 'logs': {'BIN_ACC': 0.923093321627316, 'LIN_EPS': -0.028655531207609126}}



In [ ]:
session.privacy_odometer()

{'total_epsilon_used': 5.731106241521825,
 'total_delta_used': 0.0200111,
 'library_costs': {'op_pandas': {'total_delta': 0.0,
   'total_epsilon': 3.399999999999999,
   'total_requests': 373},
  'op_tensorflow': {'total_delta': 0.0200111,
   'total_epsilon': 2.331106241521822,
   'total_requests': 5}},
 'dataset_costs': {'Heart Disease Prediction Hackathon Dataset': {'delta': 0.0200111,
   'eps': 5.731106241521825}}}

In [ ]:
%%ag
dp_model.fit(x=data_loader, epochs=5, target_delta=1) #1-e5 ? earlystop  '1 olabilir  - değiştirilebilir accuracy e göre 10 '20
x_test_scaler = standard_scaler(PrivateDataFrame(x_test), eps=2) #epsilon
y_pred = dp_model.predict(x_test_scaler, label_columns=["output"])
y_pred["output"] = y_pred["output"].map(f, output_bounds=(0, 1))
result = submit_predictions(y_pred)

Epoch 1/5

4000/4000 - 59s - loss: 0.0043 - accuracy: 0.9986 - 59s/epoch - 15ms/step

Epoch 2/5

4000/4000 - 60s - loss: 0.0046 - accuracy: 0.9988 - 60s/epoch - 15ms/step

Epoch 3/5

4000/4000 - 59s - loss: 0.0048 - accuracy: 0.9984 - 59s/epoch - 15ms/step

Epoch 4/5

4000/4000 - 60s - loss: 0.0042 - accuracy: 0.9984 - 60s/epoch - 15ms/step

Epoch 5/5

4000/4000 - 60s - loss: 0.0045 - accuracy: 0.9985 - 60s/epoch - 15ms/step

63/63 [==============================] - 0s 5ms/step

score: {'leaderboard': 0.9696552415340732, 'logs': {'BIN_ACC': 0.9983107727416823, 'LIN_EPS': -0.028655531207609126}}

